In [ ]:
//Load LDA model or the topic distributions
// data format: org.apache.spark.sql.DataFrame = [genome:string, label:string, id:long]
val k_mers_df_train = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/data_train")
val k_mers_df_test = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/data_test")

// data format: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector] the vector part contains the topic distributions
val trainingData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_train")
val testData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_test")


  

>     k_mers_df_train: org.apache.spark.sql.DataFrame = [genome: string, label: string ... 1 more field]
>     k_mers_df_test: org.apache.spark.sql.DataFrame = [genome: string, label: string ... 1 more field]
>     trainingData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector]
>     testData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector]

In [ ]:
val testData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_test_tmp")
val trainingData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_train_tmp")


  

>     testData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector]
>     trainingData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector]

In [ ]:
testData = spark.read.parquet("dbfs:/FileStore/shared_uploads/caylak@kth.se/topic_dist_test")


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

firstelement=udf(lambda v:float(v[0]),FloatType())
testData.select(firstelement('_2')).show()

  

>     +------------+
>     |<lambda>(_2)|
>     +------------+
>     | 0.098783955|
>     |  0.09921763|
>     |  0.09954901|
>     |  0.09979748|
>     |  0.09992802|
>     |  0.09876957|
>     |  0.10090187|
>     |  0.10046894|
>     |  0.10041641|
>     | 0.099867895|
>     |  0.10105545|
>     | 0.099601574|
>     |  0.09951745|
>     |  0.10045864|
>     |  0.09878429|
>     |   0.0993379|
>     |  0.10007749|
>     |  0.10007781|
>     |  0.09985662|
>     |   0.1001038|
>     +------------+
>     only showing top 20 rows

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
val mergedTrainingData = trainingData.join(k_mers_df_train,trainingData("_1") === k_mers_df_train("id"),"inner")

  

>     mergedTrainingData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 3 more fields]

In [ ]:
val mergedTestData = testData.join(k_mers_df_test,testData("_1") === k_mers_df_test("id"),"inner")

  

>     mergedTestData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 3 more fields]

In [ ]:
mergedTrainingData.show(5)

  

>     +-----+--------------------+--------------------+------------+-----+
>     |   _1|                  _2|              genome|       label|   id|
>     +-----+--------------------+--------------------+------------+-----+
>     |23492|[0.05408791370328...|AGA GAT ATC TCT C...|northamerica|23492|
>     |23506|[0.05114661180022...|AGA GAT ATC TCT C...|northamerica|23506|
>     |25084|[0.04381792648281...|TAA AAA AAG AGG G...|northamerica|25084|
>     |23772|[0.04997759657032...|AAA AAG AGG GGT G...|northamerica|23772|
>     |23870|[0.05116818178894...|ACC CCT CTT TTC T...|northamerica|23870|
>     +-----+--------------------+--------------------+------------+-----+
>     only showing top 5 rows

In [ ]:
mergedTestData.show(5)

  

>     +----+--------------------+--------------------+-------+----+
>     |  _1|                  _2|              genome|  label|  id|
>     +----+--------------------+--------------------+-------+----+
>     |1277|[0.05003667961450...|ACT CTT TTT TTC T...|oceania|1277|
>     |1224|[0.05004427388980...|ACT CTT TTT TTC T...|oceania|1224|
>     |2173|[0.05003328020089...|AGA GAT ATC TCT C...|oceania|2173|
>     | 541|[0.04996226960916...|CTC TCT CTT TTG T...|oceania| 541|
>     |1371|[0.05007064750934...|AGA GAT ATC TCT C...|oceania|1371|
>     +----+--------------------+--------------------+-------+----+
>     only showing top 5 rows

In [ ]:
//Scatter plots.  Hopefully showing some clustering depending on origin
display(mergedTrainingData.sample(false,0.5))

In [ ]:
val labelIndexer = 
              new StringIndexer().setInputCol("label").setOutputCol("label_id").fit(mergedTrainingData)

val dataIndexed = labelIndexer.transform(mergedTrainingData)

  

>     labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_f99ba8d20698, handleInvalid=error
>     dataIndexed: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 4 more fields]

In [ ]:
dataIndexed.show(5)

  

>     +-----+--------------------+--------------------+------------+-----+--------+
>     |   _1|                  _2|              genome|       label|   id|label_id|
>     +-----+--------------------+--------------------+------------+-----+--------+
>     |23492|[0.04749631516115...|AGA GAT ATC TCT C...|northamerica|23492|     0.0|
>     |23506|[0.05134092641005...|AGA GAT ATC TCT C...|northamerica|23506|     0.0|
>     |25084|[0.04625148803720...|TAA AAA AAG AGG G...|northamerica|25084|     0.0|
>     |23772|[0.05483904343166...|AAA AAG AGG GGT G...|northamerica|23772|     0.0|
>     |23870|[0.04880355486142...|ACC CCT CTT TTC T...|northamerica|23870|     0.0|
>     +-----+--------------------+--------------------+------------+-----+--------+
>     only showing top 5 rows

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.ml._


// A UDF to convert VectorUDT to ArrayType
val vecToArray = udf( (xs: linalg.Vector) => xs.toArray )


  

>     import org.apache.spark.sql.functions._
>     import org.apache.spark.ml._
>     vecToArray: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$12304/331998166@75320c6c,ArrayType(DoubleType,false),List(Some(class[value[0]: vector])),None,true,true)

In [ ]:
val dfArr = mergedTrainingData.withColumn("featuresArr" , vecToArray($"_2") )


  

>     dfArr: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 4 more fields]

In [ ]:
dfArr.show(5)

  

>     +-----+--------------------+--------------------+------------+-----+--------------------+
>     |   _1|                  _2|              genome|       label|   id|         featuresArr|
>     +-----+--------------------+--------------------+------------+-----+--------------------+
>     |23160|[0.05342589253487...|AGA GAT ATC TCT C...|northamerica|23160|[0.05342589253487...|
>     |23163|[0.05262670479927...|AGA GAT ATC TCT C...|northamerica|23163|[0.05262670479927...|
>     |24655|[0.05171709424015...|ATA TAC ACC CCT C...|northamerica|24655|[0.05171709424015...|
>     |25145|[0.05267071740042...|TAA AAA AAG AGG G...|northamerica|25145|[0.05267071740042...|
>     |23301|[0.04991413377526...|TCC CCC CCA CAG A...|northamerica|23301|[0.04991413377526...|
>     +-----+--------------------+--------------------+------------+-----+--------------------+
>     only showing top 5 rows

In [ ]:
val elements = Array("f1", "f2", "f3", "f4", "f5", "f6","f7", "f8", "f9","f10")
val sqlExpr = elements.zipWithIndex.map{ case (alias, idx) => col("featuresArr").getItem(idx).as(alias) }


  

>     elements: Array[String] = Array(f1, f2, f3, f4, f5, f6, f7, f8, f9, f10)
>     sqlExpr: Array[org.apache.spark.sql.Column] = Array(featuresArr[0] AS `f1`, featuresArr[1] AS `f2`, featuresArr[2] AS `f3`, featuresArr[3] AS `f4`, featuresArr[4] AS `f5`, featuresArr[5] AS `f6`, featuresArr[6] AS `f7`, featuresArr[7] AS `f8`, featuresArr[8] AS `f9`, featuresArr[9] AS `f10`)

In [ ]:
val df_feats = dfArr.select((col("label") +: sqlExpr) : _*)

  

>     df_feats: org.apache.spark.sql.DataFrame = [label: string, f1: double ... 9 more fields]

In [ ]:
import org.apache.spark.sql.functions.rand
display(df_feats.sample(true,0.5).orderBy(rand()))

  

[TABLE]

Truncated to 30 rows

In [ ]:
import org.apache.spark.ml.feature.{StringIndexer,VectorAssembler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.LabeledPoint
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.linalg.Vector

val transformers = Array(
              new StringIndexer().setInputCol("label").setOutputCol("label_id"))
             // new VectorAssembler()
             //        .setInputCols(Array("topic_1","topic_2","topic_3"))
             //         .setOutputCol("features"))

// Train a RandomForest model.
val rf = new RandomForestClassifier() 
              .setLabelCol("label_id")
              .setFeaturesCol("_2")
              .setNumTrees(10)
              .setFeatureSubsetStrategy("auto")
              .setImpurity("gini")
              .setMaxDepth(20)
              .setMaxBins(32)
              .setSeed(12345)

val model = new Pipeline().setStages(transformers :+ rf).fit(mergedTrainingData)

In [ ]:
val predictionsOnTestData = model.transform(mergedTestData)
predictionsOnTestData.show(5)

  

>     +----+--------------------+--------------------+-------+----+--------+--------------------+--------------------+----------+
>     |  _1|                  _2|              genome|  label|  id|label_id|       rawPrediction|         probability|prediction|
>     +----+--------------------+--------------------+-------+----+--------+--------------------+--------------------+----------+
>     |1277|[0.05003667961450...|ACT CTT TTT TTC T...|oceania|1277|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1224|[0.05004427388980...|ACT CTT TTT TTC T...|oceania|1224|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2173|[0.05003328020089...|AGA GAT ATC TCT C...|oceania|2173|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 541|[0.04996226960916...|CTC TCT CTT TTG T...|oceania| 541|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1371|[0.05007064750934...|AGA GAT ATC TCT C...|oceania|1371|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     +----+--------------------+--------------------+-------+----+--------+--------------------+--------------------+----------+
>     only showing top 5 rows
>
>     predictionsOnTestData: org.apache.spark.sql.DataFrame = [_1: bigint, _2: vector ... 7 more fields]

In [ ]:
predictionsOnTestData.show(500)

  

>     +----+--------------------+--------------------+-------+----+--------+--------------------+--------------------+----------+
>     |  _1|                  _2|              genome|  label|  id|label_id|       rawPrediction|         probability|prediction|
>     +----+--------------------+--------------------+-------+----+--------+--------------------+--------------------+----------+
>     |1277|[0.05003667961450...|ACT CTT TTT TTC T...|oceania|1277|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1224|[0.05004427388980...|ACT CTT TTT TTC T...|oceania|1224|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2173|[0.05003328020089...|AGA GAT ATC TCT C...|oceania|2173|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 541|[0.04996226960916...|CTC TCT CTT TTG T...|oceania| 541|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1371|[0.05007064750934...|AGA GAT ATC TCT C...|oceania|1371|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     | 270|[0.05001530530935...|CTT TTG TGT GTA T...|oceania| 270|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 720|[0.05004867023285...|ACT CTT TTT TTC T...|oceania| 720|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1463|[0.05007181065576...|ACT CTT TTT TTC T...|oceania|1463|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1202|[0.05005893541687...|ACT CTT TTT TTC T...|oceania|1202|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 278|[0.05001747061401...|CTC TCT CTT TTG T...|oceania| 278|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1919|[0.05002498727166...|CTT TTG TGT GTA T...|oceania|1919|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1532|[0.05004324696888...|CTT TTG TGT GTA T...|oceania|1532|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 296|[0.05002442831755...|CTT TTG TGT GTA T...|oceania| 296|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1447|[0.05009676266941...|CTT TTT TTC TCG C...|oceania|1447|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1537|[0.05002770471032...|CTT TTT TTC TCG C...|oceania|1537|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |  54|[0.05005781258609...|TTT TTC TCG CGA G...|oceania|  54|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1594|[0.05006728859284...|CTC TCT CTT TTG T...|oceania|1594|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 287|[0.05004649043162...|CTC TCT CTT TTG T...|oceania| 287|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1738|[0.05003517850042...|ACT CTT TTT TTC T...|oceania|1738|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 299|[0.05006724126238...|CTT TTG TGT GTA T...|oceania| 299|     1.0|[5.53184510723815...|[0.55318451072381...|       0.0|
>     | 857|[0.05002618201801...|ACT CTT TTT TTC T...|oceania| 857|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 564|[0.05001635153721...|ACT CTT TTT TTC T...|oceania| 564|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1497|[0.05009264042893...|ACT CTT TTT TTC T...|oceania|1497|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     | 237|[0.05004118971029...|CTT TTG TGT GTA T...|oceania| 237|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1636|[0.05002699236107...|CTT TTT TTC TCG C...|oceania|1636|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 241|[0.05005701345409...|CTT TTT TTC TCG C...|oceania| 241|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 530|[0.05003585603539...|CTT TTT TTC TCG C...|oceania| 530|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 486|[0.05005172464996...|ACT CTT TTT TTC T...|oceania| 486|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1050|[0.05000973537680...|CTT TTG TGT GTA T...|oceania|1050|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1873|[0.04999713560819...|ACT CTT TTT TTC T...|oceania|1873|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 677|[0.05000635584686...|TAC ACC CCT CTT T...|oceania| 677|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1887|[0.05007936916069...|ACT CTT TTT TTC T...|oceania|1887|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     |1041|[0.05004307588274...|CTC TCT CTT TTG T...|oceania|1041|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1427|[0.05001356258599...|CTT TTT TTC TCG C...|oceania|1427|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 532|[0.05000545399873...|TAC ACC CCT CTT T...|oceania| 532|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1329|[0.05002631728788...|AAC ACT CTT TTT T...|oceania|1329|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1460|[0.04999929318051...|AAC ACT CTT TTT T...|oceania|1460|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 830|[0.05002405877252...|ACT CTT TTT TTC T...|oceania| 830|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |2033|[0.05002042048800...|ACT CTT TTT TTC T...|oceania|2033|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 853|[0.05003421278230...|GTT TTC TCT CTC T...|oceania| 853|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1200|[0.05003157106541...|TCC CCC CCA CAG A...|oceania|1200|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 399|[0.05004065756018...|ACT CTT TTT TTC T...|oceania| 399|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1445|[0.05005408849959...|ACT CTT TTT TTC T...|oceania|1445|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1052|[0.05003918518231...|ATC TCT CTC TCT C...|oceania|1052|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1458|[0.04999461162624...|CTT TTG TGT GTA T...|oceania|1458|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 844|[0.05004573279578...|ACT CTT TTT TTC T...|oceania| 844|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 202|[0.05004309893812...|CTT TTT TTC TCG C...|oceania| 202|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 126|[0.04999771675630...|TTT TTC TCG CGA G...|oceania| 126|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1567|[0.05004647004477...|ACT CTT TTT TTC T...|oceania|1567|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 624|[0.05006469896356...|ACT CTT TTT TTC T...|oceania| 624|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1899|[0.05001671873537...|ACT CTT TTT TTC T...|oceania|1899|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 325|[0.05007203982259...|CTT TTT TTA TAA A...|oceania| 325|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     |2080|[0.05004014427252...|ACT CTT TTT TTC T...|oceania|2080|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1508|[0.04997608259000...|ACT CTT TTT TTC T...|oceania|1508|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1766|[0.05002346863933...|ACT CTT TTT TTC T...|oceania|1766|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 691|[0.05000970746293...|CTC TCT CTT TTG T...|oceania| 691|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1415|[0.05003405666427...|CTC TCT CTT TTG T...|oceania|1415|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1632|[0.05002536045701...|CTC TCT CTT TTG T...|oceania|1632|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 264|[0.05002472611180...|CTC TCT CTT TTG T...|oceania| 264|     1.0|[6.28235954749702...|[0.62823595474970...|       0.0|
>     |2196|[0.05001789806236...|ACT CTT TTT TTC T...|oceania|2196|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1343|[0.05003822301514...|ACT CTT TTT TTC T...|oceania|1343|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1559|[0.05005794201278...|TTC TCG CGA GAT A...|oceania|1559|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |2182|[0.05004053064113...|ACT CTT TTT TTC T...|oceania|2182|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 902|[0.05002264812529...|ACT CTT TTT TTC T...|oceania| 902|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 190|[0.05006232237887...|CTT TTG TGT GTA T...|oceania| 190|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |2098|[0.04996638919839...|CTT TTT TTC TCG C...|oceania|2098|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1967|[0.05002045690072...|CTT TTT TTC TCG C...|oceania|1967|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 343|[0.05000579122365...|CTT TTT TTC TCG C...|oceania| 343|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 275|[0.05005883746513...|TCG CGA GAT ATC T...|oceania| 275|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1002|[0.04997220370638...|AAG AGG GGT GTT T...|oceania|1002|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1728|[0.05004833800714...|ACT CTT TTT TTC T...|oceania|1728|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 936|[0.05001604796090...|CAA AAC ACT CTT T...|oceania| 936|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1956|[0.05000590822231...|ACT CTT TTT TTC T...|oceania|1956|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 417|[0.05003239642682...|ACT CTT TTT TTC T...|oceania| 417|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 388|[0.05004813004947...|CTC TCT CTT TTG T...|oceania| 388|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1710|[0.05007597459183...|TAA AAA AAG AGG G...|oceania|1710|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1256|[0.05000926079546...|ACT CTT TTT TTC T...|oceania|1256|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1283|[0.05004139369128...|CCA CAA AAC ACT C...|oceania|1283|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1549|[0.05004228924713...|CTT TTG TGT GTA T...|oceania|1549|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1240|[0.05005369708764...|CTT TTT TTC TCG C...|oceania|1240|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1128|[0.04998557526527...|ACT CTT TTT TTC T...|oceania|1128|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 696|[0.05005005355208...|CTC TCT CTT TTG T...|oceania| 696|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  43|[0.05003972948316...|CTT TTT TTC TCG C...|oceania|  43|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2077|[0.04995759952726...|TTC TCC CCC CCA C...|oceania|2077|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2012|[0.05006091078024...|CTT TTT TTC TCG C...|oceania|2012|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 766|[0.05003104551070...|AAC ACT CTT TTT T...|oceania| 766|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1924|[0.05003484130626...|ACT CTT TTT TTC T...|oceania|1924|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1362|[0.05004414984022...|CTT TTG TGT GTA T...|oceania|1362|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  31|[0.05002851023390...|ACT CTT TTT TTC T...|oceania|  31|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1434|[0.05001756426515...|AGA GAT ATC TCT C...|oceania|1434|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 395|[0.05001478978715...|ACT CTT TTT TTC T...|oceania| 395|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2130|[0.05001807855140...|AGA GAT ATC TCT C...|oceania|2130|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1940|[0.05005083196102...|CTT TTT TTC TCG C...|oceania|1940|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 768|[0.04993285445169...|TTC TCC CCC CCA C...|oceania| 768|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1069|[0.05001022239015...|ACT CTT TTT TTC T...|oceania|1069|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 595|[0.05006513240062...|CTC TCT CTT TTG T...|oceania| 595|     1.0|[5.53184510723815...|[0.55318451072381...|       0.0|
>     | 596|[0.04999162576341...|GTT TTT TTA TAT A...|oceania| 596|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1918|[0.05002413293836...|AAG AGG GGT GTT T...|oceania|1918|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1141|[0.05003377079217...|AGA GAT ATC TCT C...|oceania|1141|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |2185|[0.04999933447055...|CTT TTT TTC TCG C...|oceania|2185|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1548|[0.04994730555951...|TCT CTT TTG TGT G...|oceania|1548|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 966|[0.05001914702237...|ACT CTT TTT TTC T...|oceania| 966|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1612|[0.05000466627464...|AGA GAT ATC TCT C...|oceania|1612|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1504|[0.05005736683026...|CTC TCT CTT TTG T...|oceania|1504|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1945|[0.05001350518487...|CTT TTT TTC TCG C...|oceania|1945|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1775|[0.04997963678447...|CTT TTT TTC TCG C...|oceania|1775|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1530|[0.05004470423030...|CTT TTG TGT GTA T...|oceania|1530|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2110|[0.05002660892654...|ACT CTT TTT TTC T...|oceania|2110|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1757|[0.05002063406256...|AGA GAT ATC TCT C...|oceania|1757|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1099|[0.05000751297215...|CTT TTT TTC TCG C...|oceania|1099|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1299|[0.05002113774886...|TAC ACC CCT CTT T...|oceania|1299|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 948|[0.05003702332084...|CTT TTT TTC TCG C...|oceania| 948|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2026|[0.04998185923856...|TTT TTA TAT ATA T...|oceania|2026|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2059|[0.05000582096589...|ACT CTT TTT TTC T...|oceania|2059|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1441|[0.04999831551973...|ACT CTT TTT TTC T...|oceania|1441|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1858|[0.05001954964920...|ACT CTT TTT TTC T...|oceania|1858|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1323|[0.05009029280113...|AGA GAT ATC TCT C...|oceania|1323|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     |1135|[0.04999497254872...|TCG CGA GAT ATC T...|oceania|1135|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1785|[0.05001423581139...|CTC TCT CTT TTG T...|oceania|1785|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1795|[0.05000592685225...|CTT TTT TTC TCG C...|oceania|1795|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 912|[0.05002846866765...|ACT CTT TTT TTC T...|oceania| 912|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 455|[0.05003229413507...|ACT CTT TTT TTC T...|oceania| 455|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 642|[0.04998435115384...|AAG AGG GGT GTT T...|oceania| 642|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  71|[0.04998376514450...|CTT TTG TGT GTA T...|oceania|  71|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 621|[0.05005288988247...|CTT TTG TGT GTA T...|oceania| 621|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1902|[0.05001712080481...|CTT TTT TTC TCG C...|oceania|1902|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2101|[0.05001986231999...|CTT TTT TTC TCG C...|oceania|2101|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 342|[0.05002623079238...|CTT TTG TGT GTA T...|oceania| 342|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 713|[0.05002333214652...|GTA TAT ATA TAC A...|oceania| 713|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  68|[0.04999347809127...|TTT TTA TAT ATA T...|oceania|  68|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1786|[0.05004820718086...|ATT TTA TAA AAA A...|oceania|1786|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  87|[0.05004288642608...|CTC TCT CTT TTG T...|oceania|  87|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  58|[0.05004351988813...|CTC TCT CTT TTG T...|oceania|  58|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1665|[0.05000004905779...|CTT TTG TGT GTA T...|oceania|1665|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1098|[0.05004770033241...|CTT TTT TTC TCG C...|oceania|1098|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1769|[0.05002380708002...|ACT CTT TTT TTC T...|oceania|1769|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 221|[0.05003480718465...|CTT TTG TGT GTA T...|oceania| 221|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1675|[0.05000669672652...|CTT TTG TGT GTA T...|oceania|1675|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 968|[0.05001081161434...|TCC CCC CCA CAG A...|oceania| 968|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2170|[0.05005446010122...|AAA AAG AGG GGT G...|oceania|2170|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 856|[0.05000897011893...|ACT CTT TTT TTC T...|oceania| 856|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1570|[0.05001994826165...|CTT TTG TGT GTA T...|oceania|1570|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1516|[0.05002986067495...|CTT TTT TTC TCG C...|oceania|1516|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 158|[0.05005125579904...|CTT TTT TTC TCG C...|oceania| 158|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1623|[0.05000794260454...|TAA AAA AAG AGG G...|oceania|1623|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  56|[0.05003910877167...|AGA GAT ATC TCT C...|oceania|  56|     1.0|[5.48278163413989...|[0.54827816341398...|       0.0|
>     | 702|[0.05001944909235...|ACT CTT TTT TTC T...|oceania| 702|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 353|[0.05002698711592...|CTC TCT CTT TTG T...|oceania| 353|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1698|[0.05000293314261...|CTT TTT TTC TCG C...|oceania|1698|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |2037|[0.05006477437398...|CTT TTG TGT GTA T...|oceania|2037|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2066|[0.04998805750506...|AAC ACA CAA AAA A...|oceania|2066|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1307|[0.05003689997766...|CTC TCT CTT TTG T...|oceania|1307|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 861|[0.05001018456127...|ACT CTT TTT TTC T...|oceania| 861|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2140|[0.05002263595127...|CTT TTT TTC TCG C...|oceania|2140|     1.0|[5.53184510723815...|[0.55318451072381...|       0.0|
>     |1821|[0.04999337101511...|TCC CCC CCA CAG A...|oceania|1821|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1453|[0.04999061739703...|AAG AGG GGT GTT T...|oceania|1453|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2107|[0.05004317084337...|ACT CTT TTT TTC T...|oceania|2107|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  79|[0.05002412909427...|AGA GAT ATC TCT C...|oceania|  79|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1342|[0.05001178919437...|GTT TTT TTA TAT A...|oceania|1342|     1.0|[6.28429513630638...|[0.62842951363063...|       0.0|
>     |1116|[0.05006424972333...|CTT TTG TGT GTA T...|oceania|1116|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |2180|[0.04997473774845...|GGT GTT TTT TTA T...|oceania|2180|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1770|[0.05002886287044...|ACT CTT TTT TTC T...|oceania|1770|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1781|[0.05003636527480...|ACT CTT TTT TTC T...|oceania|1781|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1888|[0.05005130770334...|ACT CTT TTT TTC T...|oceania|1888|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2165|[0.05007927281560...|ACT CTT TTT TTC T...|oceania|2165|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     | 246|[0.05001270010978...|CTC TCT CTT TTG T...|oceania| 246|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1190|[0.05003328165531...|CTT TTG TGT GTA T...|oceania|1190|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 139|[0.05001712253841...|CTT TTT TTC TCG C...|oceania| 139|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1196|[0.05001082250427...|TTT TTA TAT ATA T...|oceania|1196|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 977|[0.04999542661702...|ACT CTT TTT TTC T...|oceania| 977|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 242|[0.05000494431290...|CTT TTG TGT GTA T...|oceania| 242|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1009|[0.05003350292303...|CTT TTT TTC TCG C...|oceania|1009|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1783|[0.05001839440434...|ACT CTT TTT TTC T...|oceania|1783|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1085|[0.05000440292879...|ACT CTT TTT TTC T...|oceania|1085|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2051|[0.05002956150224...|CTC TCT CTT TTG T...|oceania|2051|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 907|[0.05001524754653...|CTT TTT TTC TCG C...|oceania| 907|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 405|[0.05002525534355...|AAC ACT CTT TTT T...|oceania| 405|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1673|[0.05002855315637...|ACT CTT TTT TTC T...|oceania|1673|     1.0|[5.99929793742683...|[0.59992979374268...|       0.0|
>     | 257|[0.05004871527221...|CTT TTT TTC TCG C...|oceania| 257|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 208|[0.05000212069692...|CTT TTT TTC TCG C...|oceania| 208|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1358|[0.04999712817664...|ACT CTT TTT TTC T...|oceania|1358|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 866|[0.05002148328200...|ACT CTT TTT TTC T...|oceania| 866|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 207|[0.05001537222493...|CTT TTG TGT GTA T...|oceania| 207|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1683|[0.05005196946830...|CTT TTT TTC TCG C...|oceania|1683|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 833|[0.05006136966948...|ACT CTT TTT TTC T...|oceania| 833|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 649|[0.05002709472952...|ACT CTT TTT TTC T...|oceania| 649|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 769|[0.05001588625070...|ACT CTT TTT TTC T...|oceania| 769|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 462|[0.04994590443435...|AGG GGT GTT TTT T...|oceania| 462|     1.0|[6.34505047292379...|[0.63450504729237...|       0.0|
>     |1625|[0.05004044635871...|ACT CTT TTT TTC T...|oceania|1625|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1474|[0.05000612298418...|ACT CTT TTT TTC T...|oceania|1474|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 599|[0.05006059880534...|ACT CTT TTT TTC T...|oceania| 599|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 714|[0.05002671700823...|CTT TTG TGT GTA T...|oceania| 714|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1363|[0.05004303417078...|ACT CTT TTT TTC T...|oceania|1363|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1700|[0.04996610344558...|ACT CTT TTT TTC T...|oceania|1700|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1845|[0.05002891076186...|AAA AAG AGG GGT G...|oceania|1845|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 404|[0.05006487273332...|ACT CTT TTT TTC T...|oceania| 404|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 896|[0.05003606523717...|ACT CTT TTT TTC T...|oceania| 896|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1583|[0.05003380080349...|CTC TCT CTT TTG T...|oceania|1583|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1909|[0.05007279120600...|CTT TTT TTC TCG C...|oceania|1909|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>
>     *** WARNING: skipped 12524 bytes of output ***
>
>     |1207|[0.05003716004433...|ACT CTT TTT TTC T...|oceania|1207|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1745|[0.04997007064682...|ACT CTT TTT TTC T...|oceania|1745|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 454|[0.04993573921540...|AAG AGG GGT GTT T...|oceania| 454|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 921|[0.05002759957044...|ACT CTT TTT TTC T...|oceania| 921|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 213|[0.05014155422795...|CTC TCT CTT TTG T...|oceania| 213|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     | 192|[0.05002859732697...|CTT TTG TGT GTA T...|oceania| 192|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1107|[0.05007089482906...|ACT CTT TTT TTC T...|oceania|1107|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1984|[0.05007320437630...|ACT CTT TTT TTC T...|oceania|1984|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1691|[0.05005038830443...|ACT CTT TTT TTC T...|oceania|1691|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1524|[0.05009637272342...|CTC TCT CTT TTG T...|oceania|1524|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     | 587|[0.05002502831371...|ACT CTT TTT TTC T...|oceania| 587|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1340|[0.04997899221472...|TTA TAA AAA AAG A...|oceania|1340|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 715|[0.05002597536088...|AGA GAT ATC TCT C...|oceania| 715|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1254|[0.05000964524064...|CCA CAG AGG GGT G...|oceania|1254|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 283|[0.05004765161392...|CTC TCT CTT TTG T...|oceania| 283|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 957|[0.05007181694447...|CTT TTG TGT GTA T...|oceania| 957|     1.0|[5.99823674743359...|[0.59982367474335...|       0.0|
>     |1227|[0.05003520296240...|CTT TTT TTC TCG C...|oceania|1227|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1890|[0.05003158367837...|CTT TTT TTC TCG C...|oceania|1890|     1.0|[5.99929793742683...|[0.59992979374268...|       0.0|
>     | 982|[0.05000117134493...|CTT TTT TTC TCG C...|oceania| 982|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 206|[0.05007636072669...|TTG TGT GTA TAG A...|oceania| 206|     1.0|[5.53078391724491...|[0.55307839172449...|       0.0|
>     | 101|[0.05003519350171...|CTC TCT CTT TTG T...|oceania| 101|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 292|[0.05003753392686...|CTT TTT TTC TCG C...|oceania| 292|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 362|[0.04999566355220...|ACT CTT TTT TTC T...|oceania| 362|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 491|[0.05000491865585...|AAC ACC CCA CAA A...|oceania| 491|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 905|[0.05001423260892...|ACT CTT TTT TTC T...|oceania| 905|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1189|[0.05005293834360...|CTC TCT CTT TTG T...|oceania|1189|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 919|[0.05000575875388...|TTT TTA TAT ATA T...|oceania| 919|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1466|[0.04999524118070...|ACT CTT TTT TTC T...|oceania|1466|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 355|[0.05007356390239...|CTT TTG TGT GTA T...|oceania| 355|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     | 295|[0.05002669090513...|CTT TTT TTC TCG C...|oceania| 295|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 826|[0.05005372884899...|AAC ACT CTT TTT T...|oceania| 826|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1747|[0.05000687641720...|ACT CTT TTT TTC T...|oceania|1747|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 855|[0.05005100311718...|ACT CTT TTT TTC T...|oceania| 855|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1056|[0.04995845190910...|ATA TAC ACC CCT C...|oceania|1056|     1.0|[5.41208040135580...|[0.54120804013558...|       0.0|
>     |  55|[0.05003587511557...|CTC TCT CTT TTG T...|oceania|  55|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 373|[0.05004352984009...|CTT TTG TGT GTA T...|oceania| 373|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 922|[0.05003766989583...|CTT TTT TTC TCG C...|oceania| 922|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2142|[0.05004351799158...|CTT TTT TTC TCG C...|oceania|2142|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 664|[0.05002005307441...|ACT CTT TTT TTC T...|oceania| 664|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 492|[0.05000040271972...|ACT CTT TTT TTC T...|oceania| 492|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 390|[0.05003379382055...|ACT CTT TTT TTC T...|oceania| 390|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 996|[0.05002393307831...|CTT TTT TTC TCG C...|oceania| 996|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1245|[0.05002956498177...|ACT CTT TTT TTC T...|oceania|1245|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1062|[0.05001487198975...|ACT CTT TTT TTC T...|oceania|1062|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  62|[0.05002019577882...|CTC TCT CTT TTG T...|oceania|  62|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 512|[0.05005604245200...|CTT TTT TTC TCG C...|oceania| 512|     1.0|[5.99736234861746...|[0.59973623486174...|       0.0|
>     | 668|[0.04999535184396...|CTT TTT TTC TCG C...|oceania| 668|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |   8|[0.05007441739996...|CTT TTT TTC TCG C...|oceania|   8|     1.0|[5.53078391724491...|[0.55307839172449...|       0.0|
>     | 182|[0.05000040741649...|CTT TTT TTC TCG C...|oceania| 182|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 658|[0.04998390498600...|TAT ATA TAC ACC C...|oceania| 658|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1120|[0.05007636035237...|AAC ACT CTT TTT T...|oceania|1120|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1058|[0.05004575377516...|ACT CTT TTT TTC T...|oceania|1058|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |2143|[0.05003561265368...|ACT CTT TTT TTC T...|oceania|2143|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1225|[0.05004102817966...|CTT TTT TTC TCG C...|oceania|1225|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1758|[0.05005169172132...|CTT TTT TTC TCG C...|oceania|1758|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 910|[0.05002050965504...|GTA TAG AGA GAT A...|oceania| 910|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1707|[0.04994170767755...|TTT TTA TAT ATA T...|oceania|1707|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 710|[0.04997320319988...|ACC CCT CTT TTC T...|oceania| 710|     1.0|[6.28429513630638...|[0.62842951363063...|       0.0|
>     |1982|[0.05005787662089...|ACT CTT TTT TTC T...|oceania|1982|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1186|[0.05007266868118...|ACT CTT TTT TTC T...|oceania|1186|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1166|[0.05001269047967...|ATC TCT CTC TCT C...|oceania|1166|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1481|[0.05000133338483...|CTT TTG TGT GTA T...|oceania|1481|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 747|[0.05003138881889...|ACT CTT TTT TTC T...|oceania| 747|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 631|[0.05005679239756...|ACT CTT TTT TTC T...|oceania| 631|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1518|[0.05005325241773...|CTT TTT TTC TCG C...|oceania|1518|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1889|[0.05003383584700...|ACT CTT TTT TTC T...|oceania|1889|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 775|[0.05001414185826...|ACT CTT TTT TTC T...|oceania| 775|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  49|[0.05002816710652...|AGA GAT ATC TCT C...|oceania|  49|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 437|[0.05002412938095...|CTC TCT CTT TTG T...|oceania| 437|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 803|[0.05002617573165...|CTT TTT TTC TCG C...|oceania| 803|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  35|[0.05003414219118...|CTT TTG TGT GTA T...|oceania|  35|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |2174|[0.05003478854155...|ACT CTT TTT TTC T...|oceania|2174|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1034|[0.04997570261313...|ACT CTT TTT TTC T...|oceania|1034|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 371|[0.05005952976788...|AGA GAT ATC TCT C...|oceania| 371|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 740|[0.04998963892255...|AGA GAT ATC TCT C...|oceania| 740|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1167|[0.05002257084339...|GTA TAA AAC ACA C...|oceania|1167|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 166|[0.05003792342383...|TTC TCG CGA GAT A...|oceania| 166|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1402|[0.05001797107504...|ACT CTT TTT TTC T...|oceania|1402|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 622|[0.05002569433487...|ACT CTT TTT TTC T...|oceania| 622|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1448|[0.05004276948148...|CTC TCT CTT TTG T...|oceania|1448|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  80|[0.05003708491897...|CTT TTT TTC TCG C...|oceania|  80|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 785|[0.05006026286087...|CTT TTT TTC TCG C...|oceania| 785|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 829|[0.05003979047386...|CTT TTT TTC TCG C...|oceania| 829|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1364|[0.04997475447097...|TAC ACC CCT CTT T...|oceania|1364|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1653|[0.05004247149713...|ACT CTT TTT TTC T...|oceania|1653|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 465|[0.05001053304306...|AGG GGT GTT TTT T...|oceania| 465|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 518|[0.04999202596789...|CTC TCT CTT TTG T...|oceania| 518|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 269|[0.05001285003411...|CTT TTT TTC TCG C...|oceania| 269|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1954|[0.05009370773208...|ACT CTT TTT TTC T...|oceania|1954|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1063|[0.05008086095307...|CTT TTG TGT GTA T...|oceania|1063|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     | 508|[0.05006049332265...|ACT CTT TTT TTC T...|oceania| 508|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1489|[0.04999058040352...|TAG AGA GAT ATC T...|oceania|1489|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 869|[0.05003229659878...|ACT CTT TTT TTC T...|oceania| 869|     1.0|[5.81684230611771...|[0.58168423061177...|       0.0|
>     |1752|[0.05005155601377...|CTT TTG TGT GTA T...|oceania|1752|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 689|[0.05003812979478...|CTT TTG TGT GTA T...|oceania| 689|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1699|[0.05002420664920...|CTT TTT TTC TCG C...|oceania|1699|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 344|[0.05003072624848...|GAT ATC TCT CTC T...|oceania| 344|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1709|[0.04999281881214...|TCC CCC CCA CAG A...|oceania|1709|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 433|[0.05006247490122...|ACT CTT TTT TTC T...|oceania| 433|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 428|[0.05000674943277...|ACT CTT TTT TTC T...|oceania| 428|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 931|[0.05003783608471...|ACT CTT TTT TTC T...|oceania| 931|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 711|[0.05009336124229...|AGA GAT ATC TCT C...|oceania| 711|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     |1350|[0.04998126802085...|ATA TAC ACC CCT C...|oceania|1350|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1076|[0.05004398401313...|ATC TCT CTC TCT C...|oceania|1076|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 503|[0.05004001615325...|CTT TTG TGT GTA T...|oceania| 503|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1301|[0.04999469408300...|CTT TTT TTC TCG C...|oceania|1301|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 360|[0.05001572077012...|CTC TCT CTT TTG T...|oceania| 360|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 495|[0.05000375987375...|TTC TCC CCC CCA C...|oceania| 495|     1.0|[6.28429513630638...|[0.62842951363063...|       0.0|
>     |1611|[0.05006723144877...|ACT CTT TTT TTC T...|oceania|1611|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1081|[0.05003835065678...|ACT CTT TTT TTC T...|oceania|1081|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 329|[0.04999515487161...|ATC TCT CTC TCT C...|oceania| 329|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1596|[0.05010705684059...|CTT TTT TTC TCG C...|oceania|1596|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1344|[0.05001196016631...|CTT TTT TTC TCG C...|oceania|1344|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1565|[0.05001537902664...|ACT CTT TTT TTC T...|oceania|1565|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 634|[0.05002037786307...|CTT TTG TGT GTA T...|oceania| 634|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |2039|[0.05003174429689...|CTT TTT TTC TCG C...|oceania|2039|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1635|[0.04998467344226...|TTT TTA TAT ATA T...|oceania|1635|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 426|[0.05006011373627...|ATC TCT CTC TCT C...|oceania| 426|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1204|[0.05004692270655...|CTC TCT CTT TTG T...|oceania|1204|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1825|[0.04999875790229...|GTT TTT TTA TAT A...|oceania|1825|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1578|[0.05001426602979...|ACT CTT TTT TTC T...|oceania|1578|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 606|[0.05003077796162...|ACT CTT TTT TTC T...|oceania| 606|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1973|[0.04996975881145...|TTT TTA TAT ATA T...|oceania|1973|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 506|[0.05001651689244...|ACT CTT TTT TTC T...|oceania| 506|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 827|[0.05004512309765...|AGA GAT ATC TCT C...|oceania| 827|     1.0|[5.99736234861746...|[0.59973623486174...|       0.0|
>     | 727|[0.05003292125561...|CTT TTT TTC TCG C...|oceania| 727|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1659|[0.05004727829935...|CTT TTT TTC TCG C...|oceania|1659|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1305|[0.04996855267841...|AAG AGS GST STT T...|oceania|1305|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 604|[0.05004378243010...|AGG GGT GTA TAA A...|oceania| 604|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 842|[0.05005383113240...|CTT TTG TGT GTA T...|oceania| 842|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1814|[0.05001963366236...|CTT TTT TTC TCG C...|oceania|1814|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1962|[0.04998495493048...|TAC ACC CCT CTT T...|oceania|1962|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 810|[0.05006091952900...|TTC TCG CGA GAT A...|oceania| 810|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |2136|[0.05003054081929...|CTT TTG TGT GTA T...|oceania|2136|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 765|[0.05000346126673...|AGA GAT ATC TCT C...|oceania| 765|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |  13|[0.05004160439096...|AGA GAT ATC TCT C...|oceania|  13|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 272|[0.05003958440322...|CTC TCT CTT TTG T...|oceania| 272|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1455|[0.05002698589668...|CTT TTG TGT GTA T...|oceania|1455|     1.0|[5.53184510723815...|[0.55318451072381...|       0.0|
>     | 305|[0.05005447889256...|GTA TAG AGA GAT A...|oceania| 305|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 302|[0.05003260869388...|AGA GAT ATC TCT C...|oceania| 302|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |  36|[0.05000530422829...|AGA GAT ATC TCT C...|oceania|  36|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |2200|[0.05002316072120...|CTT TTT TTC TCG C...|oceania|2200|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |2034|[0.05000000101231...|CTT TTT TTC TCG C...|oceania|2034|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 984|[0.05001096683825...|TCG CGA GAT ATC T...|oceania| 984|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  75|[0.05004393189159...|TTC TCG CGA GAT A...|oceania|  75|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1053|[0.05003094637193...|CTT TTT TTC TCG C...|oceania|1053|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1173|[0.05001094642218...|ACT CTT TTT TTC T...|oceania|1173|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 973|[0.05005966684834...|ACT CTT TTT TTC T...|oceania| 973|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1370|[0.05001326789772...|ACT CTT TTT TTC T...|oceania|1370|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1978|[0.05005334785073...|ACT CTT TTT TTC T...|oceania|1978|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     | 314|[0.05003554077664...|CTT TTG TGT GTA T...|oceania| 314|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1886|[0.05001466781751...|ACT CTT TTT TTC T...|oceania|1886|     1.0|[6.28429513630638...|[0.62842951363063...|       0.0|
>     |2020|[0.04997860637047...|TAA AAC ACA CAA A...|oceania|2020|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1661|[0.05003981739018...|AGA GAT ATC TCT C...|oceania|1661|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1613|[0.05003461309574...|CTT TTT TTC TCG C...|oceania|1613|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 140|[0.05005127588574...|CTT TTT TTC TCG C...|oceania| 140|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 298|[0.05003206099303...|CTT TTT TTC TCG C...|oceania| 298|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 607|[0.04998434947497...|CCA CAG AGG GGT G...|oceania| 607|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1615|[0.05009844931448...|CTT TTG TGT GTA T...|oceania|1615|     1.0|[5.99823674743359...|[0.59982367474335...|       0.0|
>     | 983|[0.05005620069910...|CTT TTT TTC TCG C...|oceania| 983|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 289|[0.05007789049920...|CTT TTT TTC TCG C...|oceania| 289|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1290|[0.04999222221408...|TAC ACC CCT CTT T...|oceania|1290|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     |1070|[0.05001960033829...|ACT CTT TTT TTC T...|oceania|1070|     1.0|[5.87759764273511...|[0.58775976427351...|       0.0|
>     |1938|[0.05000379413604...|CTT TTT TTC TCG C...|oceania|1938|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1148|[0.05001028867729...|CTT TTT TTC TCG C...|oceania|1148|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 913|[0.05004197580700...|CTT TTT TTC TCG C...|oceania| 913|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 860|[0.04997644043374...|ACT CTT TTT TTC T...|oceania| 860|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  14|[0.04999851983449...|CTT TTG TGT GTA T...|oceania|  14|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1162|[0.05002763864150...|GGT GTA TAA AAC A...|oceania|1162|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1865|[0.05002784080650...|CTT TTT TTC TCG C...|oceania|1865|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1311|[0.05004235759150...|ACT CTT TTT TTC T...|oceania|1311|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 864|[0.05005072229624...|ACT CTT TTT TTC T...|oceania| 864|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 598|[0.05003246446481...|CTC TCT CTT TTG T...|oceania| 598|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1869|[0.05003540496676...|CTT TTT TTC TCG C...|oceania|1869|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1170|[0.04999046259780...|TTC TCC CCC CCA C...|oceania|1170|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 791|[0.04994093540679...|GTA TAA AAC ACA C...|oceania| 791|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1932|[0.04998190758974...|TAT ATA TAC ACC C...|oceania|1932|     1.0|[6.28235954749702...|[0.62823595474970...|       0.0|
>     |1914|[0.05005604243841...|CTT TTG TGT GTA T...|oceania|1914|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 194|[0.05007891074993...|CTT TTG TGT GTA T...|oceania| 194|     1.0|[5.53078391724491...|[0.55307839172449...|       0.0|
>     |1917|[0.05003895148806...|CTT TTG TGT GTA T...|oceania|1917|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |  21|[0.04994098177219...|ATC TCT CTC TCT C...|oceania|  21|     1.0|[6.39407008076693...|[0.63940700807669...|       0.0|
>     | 881|[0.05002967361069...|CTT TTT TTC TCG C...|oceania| 881|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1216|[0.05003006871128...|ACT CTT TTT TTC T...|oceania|1216|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |1907|[0.05003977603863...|AAG AGG GGT GTT T...|oceania|1907|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1470|[0.05003882094523...|CTC TCT CTT TTG T...|oceania|1470|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 129|[0.05002011181270...|CTT TTG TGT GTA T...|oceania| 129|     1.0|[5.92661725057825...|[0.59266172505782...|       0.0|
>     |1333|[0.05004490966954...|AGA GAT ATC TCT C...|oceania|1333|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 651|[0.05009012118589...|CTC TCT CTT TTG T...|oceania| 651|     1.0|[5.53078391724491...|[0.55307839172449...|       0.0|
>     | 393|[0.05000811028008...|ACT CTT TTT TTC T...|oceania| 393|     1.0|[6.28429513630638...|[0.62842951363063...|       0.0|
>     | 493|[0.05003159126018...|ACT CTT TTT TTC T...|oceania| 493|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 627|[0.05003166604143...|ACT CTT TTT TTC T...|oceania| 627|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     | 868|[0.05001806767457...|ACT CTT TTT TTC T...|oceania| 868|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 310|[0.05009552448382...|ATC TCT CTC TCT C...|oceania| 310|     1.0|[5.53078391724491...|[0.55307839172449...|       0.0|
>     |1030|[0.05004452766469...|CTT TTT TTC TCG C...|oceania|1030|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     |1881|[0.05008862010462...|ACT CTT TTT TTC T...|oceania|1881|     1.0|[5.53271950605428...|[0.55327195060542...|       0.0|
>     |1579|[0.05000685919365...|CTC TCT CTT TTG T...|oceania|1579|     1.0|[5.53184510723815...|[0.55318451072381...|       0.0|
>     | 223|[0.05008544365810...|CTC TCT CTT TTG T...|oceania| 223|     1.0|[5.06526667586560...|[0.50652666758656...|       0.0|
>     |1320|[0.05003015671029...|CTT TTT TTC TCG C...|oceania|1320|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     |  81|[0.05003647534741...|CTT TTT TTC TCG C...|oceania|  81|     1.0|[5.92855283938762...|[0.59285528393876...|       0.0|
>     | 878|[0.05005039133612...|CTT TTT TTC TCG C...|oceania| 878|     1.0|[5.46110000919894...|[0.54611000091989...|       0.0|
>     +----+--------------------+--------------------+-------+----+--------+--------------------+--------------------+----------+
>     only showing top 500 rows

In [ ]:
val predicitonsAndLabels = predictionsOnTestData.select("label", "prediction")

  

>     predicitonsAndLabels: org.apache.spark.sql.DataFrame = [label: double, prediction: double]

In [ ]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label_id")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictionsOnTestData)
println(s"Test Error = ${(1.0 - accuracy)}")

  

>     Test Error = 0.4179882916444918
>     import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
>     evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = MulticlassClassificationEvaluator: uid=mcEval_be7167013f36, metricName=accuracy, metricLabel=0.0, beta=1.0, eps=1.0E-15
>     accuracy: Double = 0.5820117083555082

In [ ]:
val predictionAndLabelsRdd = predictionsOnTestData.select("prediction", "label_id").as[(Double,Double)].rdd

  

>     predictionAndLabelsRdd: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[498248] at rdd at command-2294440354339436:1

In [ ]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics

val metricsMulti = new MulticlassMetrics(predictionAndLabelsRdd)

  

>     import org.apache.spark.mllib.evaluation.MulticlassMetrics
>     metricsMulti: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@2767e4e4

In [ ]:
println("Confusion matrix:")

println(metricsMulti.confusionMatrix)

val accuracy = metricsMulti.accuracy

println("Summary Statistics")

println(s"Accuracy = $accuracy")

  

>     Confusion matrix:
>     5468.0  0.0  0.0  0.0  0.0  0.0  
>     2891.0  0.0  0.0  0.0  0.0  0.0  
>     516.0   0.0  0.0  0.0  0.0  0.0  
>     393.0   0.0  0.0  0.0  0.0  0.0  
>     96.0    0.0  0.0  0.0  0.0  0.0  
>     31.0    0.0  0.0  0.0  0.0  0.0  
>     Summary Statistics
>     Accuracy = 0.5820117083555082
>     accuracy: Double = 0.5820117083555082